In [ ]:
import quantstats as qs
import pandas as pd
from datetime import datetime
import backtrader as bt
import matplotlib.pyplot as plt 
import tushare as ts
import quantstats as qs
## test
qs.extend_pandas()

#plt.rcParams["font.sans-serif"] = ["SimHei"]
#plt.rcParams["axes.unicode_minus"] = False

In [ ]:
# 按股票代码，依次加载数据
for stock in datas['sec_code'].unique():
    # 日期对齐, 获取回测区间所有交易日
    data = pd.DataFrame(index=datas.index.unique())
    df = datas.query(f"sec_code=='{stock}'")[['open','high','low','close','volume','openinterest']] 
    data_ = pd.merge(data, df, left_index=True, right_index=True, how='left')
    # 缺失值处理
    data_.loc[:,['volume','openinterest']] = data_.loc[:,['volume','openinterest']].fillna(0)
    data_.loc[:,['open','high','low','close','EP','ROE']] = data_.loc[:,['open','high','low','close']].fillna(method='pad')
    data_.loc[:,['open','high','low','close','EP','ROE']] = data_.loc[:,['open','high','low','close']].fillna(0.0000001)

    # 导入数据
    datafeed = PandasData_more(dataname=data_,
                            fromdate = datetime.datetime(2019,1,31),
                            todate= datetime.datetime(2021,8,31),
                            timeframe = bt.TimeFrame.Months
                            )
    cerebro.adddata(datafeed,name=stock)
    #print(f"{stock} Done!")

......
# 获得收益率时序
ret = pd.Series(result[0].analyzers._TimeReturn.get_analysis())

In [4]:
ts.set_token('5d3c09f16bc3ad409898ff6cfc10aadeeae28ba23aa240c047dd125c')
pro = ts.pro_api()
hs300 = pro.index_weight(index_code='000300.SH', start_date='20190101', end_date='20210831')
hs300.head()

Exception: 抱歉，您没有接口访问权限，权限的具体详情访问：https://tushare.pro/document/1?doc_id=108。

In [44]:
ts.set_token('5d3c09f16bc3ad409898ff6cfc10aadeeae28ba23aa240c047dd125c')
pro = ts.pro_api()

start_date = '20240307';end_date = '20250410';code = '600519.SH'

def get_data(code = code,start_date = start_date,end_date = end_date):
    df = pro.daily(ts_code = code,start_date = start_date,end_date = end_date)
    df = df.iloc[::-1] #序列倒转
    df.index = pd.to_datetime(df.trade_date) #index改为时间格式
    df['openinterest'] = 0 #设置Openinterest
    df = df[['open','high','low','close','vol','openinterest']]
    df.rename(columns={'vol': 'volume'}, inplace=True)
    return df

stock_df = get_data()
print(stock_df.tail())
data = bt.feeds.PandasData(dataname=stock_df,fromdate=pd.to_datetime(start_date),todate=pd.to_datetime(end_date))

               open     high      low    close     volume  openinterest
trade_date                                                             
2025-04-03  1530.00  1586.00  1529.01  1568.88   35480.50             0
2025-04-07  1520.01  1536.85  1462.00  1500.00  101952.67             0
2025-04-08  1513.00  1545.00  1495.00  1545.00   73876.49             0
2025-04-09  1525.02  1556.80  1520.55  1541.04   55630.04             0
2025-04-10  1550.99  1555.00  1528.00  1549.00   38620.21             0


In [39]:
class lab(bt.Strategy):
    params = dict(
        pfast=10,  # period for the fast moving average
        pslow=30   # period for the slow moving average
    )
    
    def __init__(self):
        self.order = None
        self.buyprice = None
        self.buycomm = None
        #self.roc = bt.ind.ROC(self.data,period=3) #period=x:(px-px[-x])/px[-x]
        sma1 = bt.ind.SMA(period=self.p.pfast)  # fast moving average
        sma2 = bt.ind.SMA(period=self.p.pslow)  # slow moving average
        self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal

    def next(self):
        #print(self.data.lines[0])
        #print(self.data.lines[0][0])
        #print(bt.num2date(self.data.lines[6][0]))
        #print(self.roc[0])
        #print(self.data.lines.close)
        #print(self.data.lines[0])
        if not self.position:  # not in the market
            if self.crossover > 0:  # if fast crosses slow to the upside
                self.buy()  # enter long

        elif self.crossover < 0:  # in the market & cross to the downside
            self.close()  # close long position


In [ ]:

cerebro = bt.Cerebro()  # create a "Cerebro" engine instance

cerebro.adddata(data)  # Add the data feed
cerebro.addstrategy(MovingAvg)  # Add the trading strategy

cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.003, annualize=True, _name='SharpeRatio')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='DrawDown')
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='AnnualReturn')
cerebro.addanalyzer(bt.analyzers.PyFolio)
cerebro.addanalyzer(bt.analyzers.TimeReturn,_name='strat_returns')

cerebro.broker.setcash(1000000)
cerebro.broker.setcommission(commission=0.0005)

cerebro.addsizer(bt.sizers.PercentSizer, percents=90)
strats = cerebro.run()  # run it all
strat = strats[0]  # run it all
print("--------------- AnnualReturn -----------------")
print(strat.analyzers.AnnualReturn.get_analysis())
print("--------------- SharpeRatio -----------------")
print(strat.analyzers.SharpeRatio.get_analysis())
print("--------------- DrawDown -----------------")
print(strat.analyzers.DrawDown.get_analysis())

pyfolio = strats[0].analyzers.getbyname('pyfolio')
returns, positions, transactions, gross_lev = pyfolio.get_pf_items()
sp500 = qs.utils.download_returns('SPY')
qs.reports.metrics(returns, benchmark=sp500)

--------------- AnnualReturn -----------------
OrderedDict({2024: -0.03771950582886152, 2025: 0.01486157782173736})
--------------- SharpeRatio -----------------
OrderedDict({'sharperatio': -0.5488271827732721})
--------------- DrawDown -----------------
AutoOrderedDict({'len': 125, 'drawdown': 11.533810667315334, 'moneydown': 127322.15792650601, 'max': AutoOrderedDict({'len': 125, 'drawdown': 15.107903143187535, 'moneydown': 166776.69552755193})})



1 Failed download:
['SPY']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
c:\Users\keyua\anaconda3\Lib\site-packages\quantstats\utils.py:279: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  .pct_change()


                    Benchmark    Strategy
------------------  -----------  ----------
Start Period        2024-03-07   2024-03-07
End Period          2025-04-10   2025-04-10
Risk-Free Rate      0.0%         0.0%
Time in Market      0.0%         45.0%

Cumulative Return   0.0%         -2.34%
CAGR﹪              0.0%         -1.49%

Sharpe              -            -0.02
Prob. Sharpe Ratio  -            49.1%
Sortino             -            -0.04
Sortino/√2          -            -0.03
Omega               0.99         0.99

Max Drawdown        -            -15.11%
Longest DD Days     -            -

Gain/Pain Ratio     -            -0.01
Gain/Pain (1M)      -            -0.02

Payoff Ratio        -            -
Profit Factor       -            0.99
Common Sense Ratio  -            0.77
CPC Index           -            -
Tail Ratio          -            0.78
Outlier Win Ratio   -            8.59
Outlier Loss Ratio  -            1.39

MTD                 0.0%         0.0%
3M                

c:\Users\keyua\anaconda3\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2246: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
c:\Users\keyua\anaconda3\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2247: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
